[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain-academy/blob/main/module-0/basics.ipynb) [![Open in LangChain Academy](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/56295530-getting-set-up-video-guide)

# LangChain Academy

Welcome to LangChain Academy! 

## Context

At LangChain, we aim to make it easy to build LLM applications. One type of LLM application you can build is an agent. There’s a lot of excitement around building agents because they can automate a wide range of tasks that were previously impossible. 

In practice though, it is incredibly difficult to build systems that reliably execute on these tasks. As we’ve worked with our users to put agents into production, we’ve learned that more control is often necessary. You might need an agent to always call a specific tool first or use different prompts based on its state. 

To tackle this problem, we’ve built [LangGraph](https://langchain-ai.github.io/langgraph/) — a framework for building agent and multi-agent applications. Separate from the LangChain package, LangGraph’s core design philosophy is to help developers add better precision and control into agent workflows, suitable for the complexity of real-world systems.

## Course Structure

The course is structured as a set of modules, with each module focused on a particular theme related to LangGraph. You will see a folder for each module, which contains a series of notebooks. A video will accompany each notebook to help walk through the concepts, but the notebooks are also stand-alone, meaning that they contain explanations and can be viewed independently of the videos. Each module folder also contains a `studio` folder, which contains a set of graphs that can be loaded into [LangGraph Studio](https://github.com/langchain-ai/langgraph-studio), our IDE for building LangGraph applications.

## Setup

Before you begin, please follow the instructions in the `README` to create an environment and install dependencies.

## Chat models

In this course, we'll be using [Chat Models](https://python.langchain.com/v0.2/docs/concepts/#chat-models), which do a few things take a sequence of messages as inputs and return chat messages as outputs. LangChain does not host any Chat Models, rather we rely on third party integrations. [Here](https://python.langchain.com/v0.2/docs/integrations/chat/) is a list of 3rd party chat model integrations within LangChain! This course will use [ChatOllama](https://python.langchain.com/docs/integrations/chat/ollama/) because it can be tested in local with all available models.


In [1]:
%%capture --no-stderr
%pip install --quiet -U langchain_ollama langchain_core langchain_community

[Here](https://python.langchain.com/v0.2/docs/how_to/#chat-models) is a useful how-to for all the things that you can do with chat models, but we'll show a few highlights below. If you've run `pip install -r requirements.txt` as noted in the README, then you've installed the `langchain-ollama` package. With this, we can instantiate our `ChatOllama` model object. 

There are [a few standard parameters](https://python.langchain.com/v0.2/docs/concepts/#chat-models) that we can set with chat models. Two of the most common are:

* `model`: the name of the model
* `temperature`: the sampling temperature

`Temperature` controls the randomness or creativity of the model's output where low temperature (close to 0) is more deterministic and focused outputs. This is good for tasks requiring accuracy or factual responses. High temperature (close to 1) is good for creative tasks or generating varied responses. 

In [2]:
from langchain_ollama import ChatOllama

llama_chat = ChatOllama(model="llama3.2", temperature=0)

Chat models in LangChain have a number of [default methods](https://python.langchain.com/v0.2/docs/concepts/#runnable-interface). For the most part, we'll be using:

* `stream`: stream back chunks of the response
* `invoke`: call the chain on an input

And, as mentioned, chat models take [messages](https://python.langchain.com/v0.2/docs/concepts/#messages) as input. Messages have a role (that describes who is saying the message) and a content property. We'll be talking a lot more about this later, but here let's just show the basics.

In [3]:
from langchain_core.messages import HumanMessage

# Create a message
msg = HumanMessage(content="Hello world", name="Lance")

# Message list
messages = [msg]

# Invoke the model with a list of messages 
llama_chat.invoke(messages)

AIMessage(content="Hello! It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-03-15T07:59:52.446773865Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3508678641, 'load_duration': 41883190, 'prompt_eval_count': 27, 'prompt_eval_duration': 408000000, 'eval_count': 25, 'eval_duration': 3057000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-ba091649-e258-4cbe-8017-82191ad6b067-0', usage_metadata={'input_tokens': 27, 'output_tokens': 25, 'total_tokens': 52})

We get an `AIMessage` response. Also, note that we can just invoke a chat model with a string. When a string is passed in as input, it is converted to a `HumanMessage` and then passed to the underlying model.


In [4]:
llama_chat.invoke("hello world")

AIMessage(content="Hello! It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-03-15T07:59:55.969126381Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3509299226, 'load_duration': 36898272, 'prompt_eval_count': 27, 'prompt_eval_duration': 241000000, 'eval_count': 25, 'eval_duration': 3230000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-f8280320-6f80-409c-8453-8db51b793366-0', usage_metadata={'input_tokens': 27, 'output_tokens': 25, 'total_tokens': 52})

The interface is consistent across all chat models and models are typically initialized once at the start up each notebooks. 

So, you can easily switch between models without changing the downstream code if you have strong preference for another provider.


## Search Tools

You'll also see [Tavily](https://tavily.com/) in the README, which is a search engine optimized for LLMs and RAG, aimed at efficient, quick, and persistent search results. As mentioned, it's easy to sign up and offers a generous free tier. Some lessons (in Module 4) will use Tavily by default but, of course, other search tools can be used if you want to modify the code for yourself.

In [11]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

In [12]:
_set_env("TAVILY_API_KEY")

TAVILY_API_KEY:  ········


In [13]:
from langchain_community.tools.tavily_search import TavilySearchResults
tavily_search = TavilySearchResults(max_results=3)
search_docs = tavily_search.invoke("What is Google?")

In [14]:
search_docs

[{'title': 'Google | History & Facts; Products & Services | Britannica Money',
  'url': 'https://www.britannica.com/money/Google-Inc',
  'content': 'Google | History & Facts; Products & Services | Britannica Money Google is an American search engine company, founded in 1998 by Sergey Brin and Larry Page. Since 2015, Google has been a subsidiary of the holding company Alphabet, Inc. More than 70% of worldwide online search requests are handled by Google, placing it at the heart of most Internet users’ experience. Two years later the company responded to the explosive growth of the mobile applications market with a $750 million deal to acquire the mobile advertising network AdMob. All of these purchases were part of Google’s effort to expand from its search engine business into advertising by combining the various firms’ databases of information in order to tailor ads to consumers’ individual preferences.',
  'score': 0.85967577},
 {'title': 'Google - Wikipedia',
  'url': 'https://en.wik